# Tweets Analysis for Baseball Game Audience

## Part 1 - Setup

Setting up Spark and fetching the project data.  


### About the data

We will use JSON data from Twitter.

This data was gathered using GWU Libraries' [Social Feed Manager](http://sfm.library.gwu.edu/) application during a recent game of the MLB World Series featuring the Los Angeles Dodgers and Houston Astros. 

The most important pieces in that metadata are:

 * It tracked tweets that mentioned "dodgers" or "astros".  Every item in this set should refer to one or the other, or both.
 * This data was not deduplicated; we may see individual items more than once.
 * Data was collected between October 29 and October 30.  Game 5 of the Series was played during this time.
 

### Fetch the data

The following files are available at:

 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json
 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_004.json
 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_005.json
 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_006.json
 

In [5]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json

--2017-11-03 11:55:58--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.20.149
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.20.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 595711407 (568M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_003.json’

9670f3399f774789b7c 100%[===================>] 568.11M  96.0MB/s    in 6.1s    

2017-11-03 11:56:05 (93.2 MB/s) - ‘9670f3399f774789b7c3e18975d25611_003.json’ saved [595711407/595711407]



In [6]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_004.json

--2017-11-03 11:56:06--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_004.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.0.171
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.0.171|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 549995846 (525M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_004.json’

9670f3399f774789b7c 100%[===================>] 524.52M  81.1MB/s    in 6.1s    

2017-11-03 11:56:12 (85.4 MB/s) - ‘9670f3399f774789b7c3e18975d25611_004.json’ saved [549995846/549995846]



In [7]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_005.json

--2017-11-03 11:56:19--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_005.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.32.106
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.32.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 530698683 (506M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_005.json’

9670f3399f774789b7c 100%[===================>] 506.11M  92.0MB/s    in 5.4s    

2017-11-03 11:56:25 (93.2 MB/s) - ‘9670f3399f774789b7c3e18975d25611_005.json’ saved [530698683/530698683]



In [8]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_006.json

--2017-11-03 11:56:34--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_006.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.162.5
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.162.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 545081593 (520M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_006.json’

9670f3399f774789b7c 100%[===================>] 519.83M  93.9MB/s    in 5.5s    

2017-11-03 11:56:39 (95.2 MB/s) - ‘9670f3399f774789b7c3e18975d25611_006.json’ saved [545081593/545081593]



In [9]:
!wc -l *.json

    100000 9670f3399f774789b7c3e18975d25611_003.json
    100000 9670f3399f774789b7c3e18975d25611_004.json
    100000 9670f3399f774789b7c3e18975d25611_005.json
    100000 9670f3399f774789b7c3e18975d25611_006.json
    400000 total


Randomly select one from these files to take a look at the data structures. 

In [85]:
!cat *.json | shuf -n 1 > example-tweet.json

In [86]:
import json
print(json.dumps(json.load(open("example-tweet.json")), indent=2))

{
  "lang": "en",
  "retweeted": false,
  "contributors": null,
  "text": "RT @astros: BREGMAN WALKS IT OFF! #ASTROSWIN 13-12! LET'S GOOOOOOOOOO https://t.co/wE5jsWV8ym",
  "truncated": false,
  "place": null,
  "in_reply_to_status_id": null,
  "retweet_count": 0,
  "id_str": "924899969785614336",
  "favorite_count": 0,
  "created_at": "Mon Oct 30 07:25:11 +0000 2017",
  "retweeted_status": {
    "lang": "en",
    "retweeted": false,
    "text": "BREGMAN WALKS IT OFF! #ASTROSWIN 13-12! LET'S GOOOOOOOOOO https://t.co/wE5jsWV8ym",
    "truncated": false,
    "place": null,
    "in_reply_to_status_id": null,
    "retweet_count": 11405,
    "id_str": "924873346252263424",
    "favorite_count": 12823,
    "created_at": "Mon Oct 30 05:39:23 +0000 2017",
    "display_text_range": [
      0,
      57
    ],
    "in_reply_to_user_id_str": null,
    "coordinates": null,
    "id": 924873346252263424,
    "filter_level": "low",
    "favorited": false,
    "reply_count": 303,
    "in_reply_to_scree

We can find several key elements in this example; the text, time, and language of the tweet, whether it was a reply to another user, the user's screen name along with their primary language and other account information like creation date, follower/friend/tweet counts, and perhaps their location. If there are hashtags, user mentions, or urls present in their tweet, they will be present in the `entities` section, but these are not present in every tweet. 

### Start up Spark

In [2]:
import os

In [3]:
os.environ['SPARK_HOME'] = '/usr/local/lib/spark'

In [4]:
import findspark

In [5]:
findspark.init()

In [6]:
from pyspark import SparkContext

In [7]:
spark = SparkContext(appName='project-03')

In [8]:
spark

<SparkContext master=local[*] appName=project-03>

In [9]:
from pyspark import SQLContext

In [10]:
sqlc = SQLContext(spark)

In [11]:
sqlc

In [12]:
tweets = sqlc.read.json("9670f3399f774789b7c3e18975d25611_*.json")

In [13]:
tweets.count()

400000

We see exactly the same number of tweets in Spark that we saw on the command line; this looks correct.

## Part 2 - User Profile and Tweets Analysis(using both DataFrames and Spark SQL)

In [14]:
tweets

DataFrame[contributors: string, coordinates: struct<coordinates:array<double>,type:string>, created_at: string, display_text_range: array<bigint>, entities: struct<hashtags:array<struct<indices:array<bigint>,text:string>>,media:array<struct<display_url:string,expanded_url:string,id:bigint,id_str:string,indices:array<bigint>,media_url:string,media_url_https:string,sizes:struct<large:struct<h:bigint,resize:string,w:bigint>,medium:struct<h:bigint,resize:string,w:bigint>,small:struct<h:bigint,resize:string,w:bigint>,thumb:struct<h:bigint,resize:string,w:bigint>>,source_status_id:bigint,source_status_id_str:string,source_user_id:bigint,source_user_id_str:string,type:string,url:string>>,symbols:array<struct<indices:array<bigint>,text:string>>,urls:array<struct<display_url:string,expanded_url:string,indices:array<bigint>,url:string>>,user_mentions:array<struct<id:bigint,id_str:string,indices:array<bigint>,name:string,screen_name:string>>>, extended_entities: struct<media:array<struct<display_

To issue SQL queries, we need to register a table based on `tweets`:

In [15]:
tweets.createOrReplaceTempView("tweets")

### Top 10 languages are most commonly used among twitter users

In [16]:
tweets.groupBy('lang').count().orderBy('count', ascending=False).show(10)

+----+------+
|lang| count|
+----+------+
|  en|346302|
|  es| 31869|
| und| 14659|
|  in|  2355|
|  fr|   997|
|  pt|   914|
|  tl|   485|
|  ht|   377|
|  nl|   347|
|  it|   285|
+----+------+
only showing top 10 rows



In [17]:
sqlc.sql("""
    SELECT lang, COUNT(*)
    FROM tweets
    GROUP BY lang
    ORDER BY COUNT(*) DESC
""").show(10)

+----+--------+
|lang|count(1)|
+----+--------+
|  en|  346302|
|  es|   31869|
| und|   14659|
|  in|    2355|
|  fr|     997|
|  pt|     914|
|  tl|     485|
|  ht|     377|
|  nl|     347|
|  it|     285|
+----+--------+
only showing top 10 rows



It appears that English and Spanish are the most common languages for tweets, and perhaps `und` means "undetermined".

### Top 10 time zones are most common among twitter users

In [18]:
tweets.groupBy('user.time_zone').count().orderBy('count', ascending=False).show(10)

+--------------------+------+
|           time_zone| count|
+--------------------+------+
|                null|166386|
|Pacific Time (US ...| 72214|
|Central Time (US ...| 63664|
|Eastern Time (US ...| 35889|
|             Arizona| 10748|
|Mountain Time (US...| 10057|
|Atlantic Time (Ca...|  4805|
|               Quito|  4397|
|             Caracas|  4218|
|         Mexico City|  3935|
+--------------------+------+
only showing top 10 rows



In [19]:
sqlc.sql("""
    SELECT user.time_zone, COUNT(*)
    FROM tweets
    GROUP BY user.time_zone
    ORDER BY COUNT(*) DESC
""").show(10)

+--------------------+--------+
|           time_zone|count(1)|
+--------------------+--------+
|                null|  166386|
|Pacific Time (US ...|   72214|
|Central Time (US ...|   63664|
|Eastern Time (US ...|   35889|
|             Arizona|   10748|
|Mountain Time (US...|   10057|
|Atlantic Time (Ca...|    4805|
|               Quito|    4397|
|             Caracas|    4218|
|         Mexico City|    3935|
+--------------------+--------+
only showing top 10 rows



We see the same result in both queries; the most common value is `null`, but among present values, US time zones dominate, led by western time zones.  This makes sense as the two teams playing are based in western time zones.  Outside of the US, Central and South American zones appear, which makes sense given the number of MLB players from these regions.

### Number of tweets mention the Dodgers, the Astros and both

In [20]:
sqlc.sql("""
    SELECT COUNT(*)
    FROM tweets
    WHERE LOWER(text) LIKE '%astros%'
""").show()

+--------+
|count(1)|
+--------+
|  246465|
+--------+



In [21]:
sqlc.sql("""
    SELECT COUNT(*)
    FROM tweets
    WHERE LOWER(text) LIKE '%dodgers%'
""").show()

+--------+
|count(1)|
+--------+
|  168218|
+--------+



In [22]:
sqlc.sql("""
    SELECT COUNT(*)
    FROM tweets
    WHERE LOWER(text) LIKE '%dodgers%'
      AND LOWER(text) LIKE '%astros%'
""").show()

+--------+
|count(1)|
+--------+
|   49545|
+--------+



From our results above, it appears that nearly 50% more tweets mention the Astros than mention the Dodgers.  At 49,545, more than 10% of the total 400,000 tweets mention both teams.

We used the `LOWER()` function to ensure case-independent matching and catch all cases.

### Team mentions by location

In [76]:
sqlc.sql("""
    SELECT user.time_zone AS tz, COUNT(*)
    FROM tweets
    WHERE LOWER(text) LIKE '%dodgers%'
      AND user.time_zone IS NOT NULL
    GROUP BY user.time_zone
    ORDER BY COUNT(*) DESC
""").show(10)

+--------------------+--------+
|                  tz|count(1)|
+--------------------+--------+
|Pacific Time (US ...|   41072|
|Central Time (US ...|   15294|
|Eastern Time (US ...|   13086|
|             Arizona|    6477|
|Mountain Time (US...|    3396|
|         Mexico City|    2512|
|             Caracas|    2123|
|              Alaska|    2064|
|               Quito|    1888|
|Atlantic Time (Ca...|    1830|
+--------------------+--------+
only showing top 10 rows



In [79]:
sqlc.sql("""
    SELECT user.time_zone AS tz, COUNT(*)
    FROM tweets
    WHERE LOWER(text) LIKE '%astros%'
      AND user.time_zone IS NOT NULL
    GROUP BY user.time_zone
    ORDER BY COUNT(*) DESC
""").show(10)

+--------------------+--------+
|                  tz|count(1)|
+--------------------+--------+
|Central Time (US ...|   49291|
|Pacific Time (US ...|   33347|
|Eastern Time (US ...|   24345|
|Mountain Time (US...|    6970|
|             Arizona|    4520|
|Atlantic Time (Ca...|    3269|
|               Quito|    2745|
|             Caracas|    2730|
|         Mexico City|    2251|
|              Hawaii|    2168|
+--------------------+--------+
only showing top 10 rows



In [80]:
sqlc.sql("""
    SELECT user.location AS location, COUNT(*)
    FROM tweets
    WHERE LOWER(text) LIKE '%dodgers%'
      AND user.location IS NOT NULL
    GROUP BY user.location
    ORDER BY COUNT(*) DESC
""").show(10)

+---------------+--------+
|       location|count(1)|
+---------------+--------+
|Los Angeles, CA|    6361|
|California, USA|    2701|
|    Houston, TX|    2362|
|    Los Angeles|    1967|
|  United States|    1578|
|     California|     866|
|     Texas, USA|     832|
|         México|     698|
|  Las Vegas, NV|     610|
|      Venezuela|     607|
+---------------+--------+
only showing top 10 rows



In [81]:
sqlc.sql("""
    SELECT user.location AS location, COUNT(*)
    FROM tweets
    WHERE LOWER(text) LIKE '%astros%'
      AND user.location IS NOT NULL
    GROUP BY user.location
    ORDER BY COUNT(*) DESC
""").show(10)

+---------------+--------+
|       location|count(1)|
+---------------+--------+
|    Houston, TX|   17639|
|     Texas, USA|    4159|
|          Texas|    2647|
|Los Angeles, CA|    2448|
|        Houston|    2133|
|  United States|    1855|
| Houston, Texas|    1852|
|     Austin, TX|    1503|
|California, USA|    1294|
|San Antonio, TX|    1269|
+---------------+--------+
only showing top 10 rows



As we might expect, we see that users mentioning the Dodgers are most frequently in the Pacific time zone, which contains California, home to the Dodgers, and that users mentioning the Astros are most frequently in the Central time zone, containing Texas, home to the Astros.  Similarly, when looking at user-report locations, Los Angeles and California are most common for Dodgers mentions, and Houston and Texas are most common for the Astros.

It is interesting to note that these terms vary quite a bit; we see both "Los Angeles, CA" and "Los Angeles", for example, as well as both "Houston, TX" and "Houston".  This is probably due to this location field being user self-reported.

We eliminate NULL values in both cases to get a clearer picture of the data present.

### Twitter users that are being replied to the most?

In [84]:
sqlc.sql("""
    SELECT in_reply_to_screen_name, COUNT(*)
    FROM tweets
    GROUP BY in_reply_to_screen_name
    ORDER BY COUNT(*) DESC
""").show(10)

+-----------------------+--------+
|in_reply_to_screen_name|count(1)|
+-----------------------+--------+
|                   null|  360299|
|                 astros|    4034|
|                Dodgers|    3297|
|                    MLB|     705|
|          stephenasmith|     332|
|               MLBONFOX|     217|
|              adevaldes|     180|
|          DodgersNation|     165|
|          rolandsmartin|     141|
|        JustinVerlander|     118|
+-----------------------+--------+
only showing top 10 rows



Nearly 90% of these tweets do not reply to any particular user, as evidenced by the NULL value at the top of this list.  Among those mentioned, it appears that the two teams themselves, as well as the league `@MLB`, are the most mentioned.  The ESPN commentator `@stephenasmith` appears frequently, as does `@JustinVerlander`, the Astros' pitcher in this game.

### Top 10 verified users that have the most followers and top 10 unverified users that have the most followers

In [97]:
sqlc.sql("""
    SELECT user.screen_name, MAX(user.followers_count) AS max_count
    FROM tweets
    WHERE user.verified == 't'
    GROUP BY user.screen_name
    ORDER BY max_count DESC
""").show(10)

+--------------+---------+
|   screen_name|max_count|
+--------------+---------+
|        cnnbrk| 53191119|
|       nytimes| 39959480|
|           CNN| 38209973|
|       Reuters| 18937529|
|       FoxNews| 16272836|
|           ABC| 12551437|
|washingtonpost| 11417638|
|   lopezdoriga|  7859821|
|           MLB|  7841255|
|           NPR|  7289619|
+--------------+---------+
only showing top 10 rows



In [99]:
sqlc.sql("""
    SELECT user.screen_name, MAX(user.followers_count) AS max_count
    FROM tweets
    WHERE user.verified == 'f'
    GROUP BY user.screen_name
    ORDER BY max_count DESC
""").show(10)

+---------------+---------+
|    screen_name|max_count|
+---------------+---------+
|Daminous_Purity|   998742|
|        chochos|   833669|
|     TexasHumor|   816301|
|  el_carabobeno|   725952|
|       PAMsLOvE|   712254|
| mlbtraderumors|   659851|
|        jilevin|   568341|
|    sun_das_ill|   559669|
|   DiegoArcos14|   544926|
|    TVCDeportes|   543095|
+---------------+---------+
only showing top 10 rows



The above shows that verified user `@cnnbrk` has the most followers, and user `@Daminous_Purity` has the most followers among unverified users.  We use the `MAX()` and `GROUP BY` to find the distinct entries for users with the most followers; we need these both because otherwise we might see multiple entries for a given user with separate follower counts.  This can arise as more people follow an account during the interval between one of their tweets and the next.

It is interesting to note that most of the popular accounts listed above appear to be media outlets.

### The most popular sets of hashtags among users with many followers and users with few followers

In [123]:
sqlc.sql("""
    SELECT LOWER(CONCAT_WS(',', SORT_ARRAY(entities.hashtags.text))) AS hashtags, COUNT(*)
    FROM tweets
    WHERE CONCAT_WS(',', entities.hashtags.text) != ''
    GROUP BY hashtags
    ORDER BY COUNT(*) DESC
""").show(10)

+--------------------+--------+
|            hashtags|count(1)|
+--------------------+--------+
|         earnhistory|   32142|
|         worldseries|   31107|
|              astros|    9895|
|            thisteam|    8413|
|             dodgers|    6461|
|               hr4hr|    4608|
|           astroswin|    4601|
|thisteam,worldseries|    4181|
|  astros,worldseries|    3753|
|thisteam,thisteam...|    3532|
+--------------------+--------+
only showing top 10 rows



This is our baseline query.  It took a few tries to get that array handling right!  This might stump a few groups.

We can use this baseline to separate users with many followers.  

In [25]:
sqlc.sql("""
    SELECT AVG(user.followers_count) AS avg_followers_count
    FROM tweets
""").show()

+-------------------+
|avg_followers_count|
+-------------------+
|       4943.6837025|
+-------------------+



From this query, we can see that the average number of followers is just under 5,000.  We've already seen that some accounts have millions of followers, so this is perhaps skewed by those outliers.  For simplicity, let's assume that 5,000 users are "many"; it is a nice round number and very few people I know actually know that many people! 

In [124]:
sqlc.sql("""
    SELECT LOWER(CONCAT_WS(',', SORT_ARRAY(entities.hashtags.text))) AS hashtags, COUNT(*)
    FROM tweets
    WHERE CONCAT_WS(',', entities.hashtags.text) != ''
    AND user.followers_count >= 5000
    GROUP BY hashtags
    ORDER BY COUNT(*) DESC
""").show(10)

+--------------------+--------+
|            hashtags|count(1)|
+--------------------+--------+
|         worldseries|    1368|
|         earnhistory|     675|
|              astros|     627|
|             dodgers|     518|
|  astros,worldseries|     244|
|            thisteam|     220|
|astros,dodgers,wo...|     219|
|        seriemundial|     190|
|      astros,dodgers|     172|
| dodgers,worldseries|     148|
+--------------------+--------+
only showing top 10 rows



In [125]:
sqlc.sql("""
    SELECT LOWER(CONCAT_WS(',', SORT_ARRAY(entities.hashtags.text))) AS hashtags, COUNT(*)
    FROM tweets
    WHERE CONCAT_WS(',', entities.hashtags.text) != ''
    AND user.followers_count < 5000
    GROUP BY hashtags
    ORDER BY COUNT(*) DESC
""").show(10)

+--------------------+--------+
|            hashtags|count(1)|
+--------------------+--------+
|         earnhistory|   31467|
|         worldseries|   29739|
|              astros|    9268|
|            thisteam|    8193|
|             dodgers|    5943|
|           astroswin|    4514|
|               hr4hr|    4491|
|thisteam,worldseries|    4038|
|  astros,worldseries|    3509|
|thisteam,thisteam...|    3480|
+--------------------+--------+
only showing top 10 rows



The most common hashtag among popular users is "worldseries", with "earnhistory" in second.  Among users with fewer followers, these are the top two again, with their position reversed.  The counts are all higher in this second set; it could be that less-followed users use fewer hashtags, but it is more likely that we simply have more tweets in our sample from less-followed users.

It is interesting to note that "astroswin" does not seem to appear in the top ten among more-followed users; perhaps they tend to be more official outlets like media agencies, and because of that, they might be less likely to tweet with the implied emotion of claiming victory.

### Analyze common words in tweet text

In [147]:
stopwords = ['a', 'an', 'and', 'for', 'i', 'in', 'is', 'it', 'of', 'on', 'rt', 'the', 'this', 'to']

In [149]:
tweets.rdd.flatMap(lambda t: t['text'].split(' ')) \
    .filter(lambda w: w != '') \
    .map(lambda w: w.lower()) \
    .filter(lambda w: w not in stopwords) \
    .map(lambda w: (w, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .takeOrdered(10, key=lambda pair: -pair[1])

[('astros', 88985),
 ('dodgers', 70845),
 ('#worldseries', 55042),
 ('@astros:', 49823),
 ('game', 45437),
 ('@astros', 40051),
 ('#thisteam', 38429),
 ('#earnhistory', 37293),
 ('#astros', 31108),
 ('@dodgers', 29941)]

I am finding the words "thisteam" and "earnhistory" to be interesting.  Perhaps they are marketing terms from the MLB, or maybe just fans from one team or another are using them.  Given our earlier findings about location and time zone, let's see whether there's a big difference in usage of these terms in each.

In [152]:
sqlc.sql("""
    SELECT user.time_zone, user.location, COUNT(*)
    FROM tweets
    WHERE LOWER(text) LIKE "%earnhistory%"
      AND user.time_zone IS NOT NULL
      AND user.location IS NOT NULL
    GROUP BY user.time_zone, user.location
    ORDER BY COUNT(*) DESC
""").show(10)

+--------------------+--------------+--------+
|           time_zone|      location|count(1)|
+--------------------+--------------+--------+
|Central Time (US ...|   Houston, TX|    1975|
|Pacific Time (US ...|   Houston, TX|     490|
|Central Time (US ...|Houston, Texas|     423|
|Eastern Time (US ...|   Houston, TX|     390|
|Central Time (US ...|         Texas|     370|
|Central Time (US ...|    Texas, USA|     360|
|Central Time (US ...|       Houston|     316|
|Central Time (US ...|    Austin, TX|     162|
|Central Time (US ...|   Houston, Tx|     152|
|Mountain Time (US...|   Houston, TX|     130|
+--------------------+--------------+--------+
only showing top 10 rows



This is a surprise!  All of the top ten time zone / location pairs with tweets using the term "earnhistory" self-identify as being in Houston, Austin, which is near Houston, or at least Texas.  This is true even among users tweeting from the Pacific and Eastern time zones!  It is clear that this term seems to be used almost entirely by Houston fans.

Let's repeat the process for "thisteam".

In [153]:
sqlc.sql("""
    SELECT user.time_zone, user.location, COUNT(*)
    FROM tweets
    WHERE LOWER(text) LIKE "%thisteam%"
      AND user.time_zone IS NOT NULL
      AND user.location IS NOT NULL
    GROUP BY user.time_zone, user.location
    ORDER BY COUNT(*) DESC
""").show(10)

+--------------------+--------------------+--------+
|           time_zone|            location|count(1)|
+--------------------+--------------------+--------+
|Pacific Time (US ...|     Los Angeles, CA|     766|
|Pacific Time (US ...|         Los Angeles|     327|
|Pacific Time (US ...|     California, USA|     195|
|Pacific Time (US ...|          California|     118|
|             Arizona|     Los Angeles, CA|      93|
|Pacific Time (US ...| Southern California|      79|
|Pacific Time (US ...|                  LA|      49|
|Pacific Time (US ...|Los Angeles, Cali...|      47|
|         Mexico City|              México|      47|
|Pacific Time (US ...|      Long Beach, CA|      47|
+--------------------+--------------------+--------+
only showing top 10 rows



Again we find a similar pattern - "thisteam" appears to be a rallying cry for Dodgers fans, as nearly all listed above are in the Pacific time zone and identify as being in Los Angeles, Southern California, or at least California.

It appears that this pair or results indicates a strong marketing strategy by each team, or at least a coalescence of fan spirit in both cases.